Here, I have taken a sample section of the csv file that we have our data on and added in any missing blocks, for now I have dropped gas columns but will try to add back in via alchemy api as demonstrated in the first few lines of code. It's a lot of data to add back in so will be simple, but painful. For now, we can run the models without wgv.

In [1]:
import numpy as np
import pandas as pd
import requests
from web3 import Web3, HTTPProvider
from nltk import flatten

In [13]:
df = pd.read_csv('../Ethdata/temp_df.csv')  # use this to load in prior CSV
df.head()

,block_number,miner_reward,gas_used,gas_price,MEV_category
0,11834049,3.722500e+17,4505938,82613218019,3.0
1,11834380,7.298672e+15,175813,41513833736,3.0
2,11834911,2.274317e+17,2696401,84346407854,3.0
3,11834918,3.235382e+16,209853,154173718335,3.0
4,11835001,2.666184e+16,77137,345642631108,3.0


In [14]:
df.drop('MEV_category',axis=1,inplace=True)  # drop any columns needed, in this instance drop prior 'MEV_category' column as we are re standardising based on blocks, not transactions

In [15]:
df['miner_reward'].describe()  # shows interquartile ranges, we use this in condition below to categorise MEV into sections

count    1.484621e+06
mean     1.403406e+17
std      9.181236e+18
min     -4.430018e+21
25%      1.104655e+16
50%      3.601383e+16
75%      1.061667e+17
max      6.000039e+21
Name: miner_reward, dtype: float64

In [16]:
df['MEV_category'] = df['miner_reward']

#using the different quartiles to define the leveles of MEV when MEV is greater than 0

a = int(list(df['miner_reward'].describe())[4])
b = int(list(df['miner_reward'].describe())[5])
c = int(list(df['miner_reward'].describe())[6])

df['MEV_category'] = df['MEV_category'].apply(lambda x: 0 if 0 > x <= a else x ) #There are some eronious negative values thrown up by mev inspect, I will label these as 0 mev
df['MEV_category'] = df['MEV_category'].apply(lambda x: 1 if 0 < x <= a else x )
df['MEV_category'] = df['MEV_category'].apply(lambda x: 2 if x <= b and x > a else x )
df['MEV_category'] = df['MEV_category'].apply(lambda x: 3 if x <= c and x > b else x )
df['MEV_category'] = df['MEV_category'].apply(lambda x: 4 if x >c else x )

In [ ]:
df2 = (df.set_index('block_number')
   .reindex(range(df.block_number.iat[0],df.block_number.iat[-1]+1), fill_value=0)
   .reset_index())   # get all blocks filled in (including 0 MEV blocks)

In [ ]:
df2.head()

,block_number,miner_reward,gas_used,gas_price,MEV_category
0,11834049,3.722500e+17,4505938,82613218019,4.0
1,11834050,0.000000e+00,0,0,0.0
2,11834051,0.000000e+00,0,0,0.0
3,11834052,0.000000e+00,0,0,0.0
4,11834053,0.000000e+00,0,0,0.0


In [ ]:
df2.drop('gas_used',axis=1,inplace=True) # drop columns for now, we can use gas later if needed

In [ ]:
df2.drop('gas_price',axis=1,inplace=True) # use 'inplace = True' to make adjustment to df itself - don't use this arguent if you just want to look at df withough the dropped column

In [ ]:
df2.head()

,block_number,miner_reward,MEV_category
0,11834049,3.722500e+17,4.0
1,11834050,0.000000e+00,0.0
2,11834051,0.000000e+00,0.0
3,11834052,0.000000e+00,0.0
4,11834053,0.000000e+00,0.0


In [ ]:
df2['MEV_category'].value_counts()

0.0    1668305
4.0     371155
2.0     371155
3.0     371155
1.0     371137
Name: MEV_category, dtype: int64

In [ ]:
df2.columns

Index(['block_number', 'miner_reward', 'MEV_category'], dtype='object')

In [ ]:
df2.drop(['miner_reward'],axis=1, inplace=True)

In [ ]:
df2 = df2.reset_index(drop=True) #lastly we just need to get rid of the index collumns because we want to use ht block number as the index

In [ ]:
df2.head()

,block_number,MEV_category
0,11834049,4.0
1,11834050,0.0
2,11834051,0.0
3,11834052,0.0
4,11834053,0.0


In [ ]:
#then we can save the data that has been created using this line. 

df2.to_csv('../Ethdata/final_df.csv') 


## Time to add the eth price volitility to the collumns to consider other factors

### Adding soem metric of gas fees. 

In [ ]:
url = "https://mainnet.gateway.pokt.network/v1/lb/62fd2e8c466be4003a7ac327/v1/query/blocktxs" # my free alchemy historical api 

In [ ]:
r = requests.post(url) # post request 

In [ ]:
# Initialize a Web3.py instance
web3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/PXDcncN7KHOchKhO4S9FzldLoXBtshEI'))

In [ ]:
block = 11834049
gas_data = []
for i in range(40):  # adjust for loop based on amount of data required
    hist = web3.eth.fee_history(newest_block = block, block_count=1000)
    gas_data.append(hist['gasUsedRatio'])
    block += 1000

In [ ]:
gas_data2 = flatten(gas_data) # change nested list into one dimensional list

In [ ]:
len(gas_data2)

40000

In [ ]:
df3 = df2.head(len(gas_data2)) # taking 10000 rows of data, so len(gas_data2) = len(df)

In [ ]:
df3['gas_used_ratio'] = gas_data2 # adding in gas column

C:\Users\sufya\AppData\Local\Temp/ipykernel_4080/3810027331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['gas_used_ratio'] = gas_data2 # adding in gas column


In [ ]:
df3.head()

,block_number,MEV_category,gas_used_ratio
0,11834049,4.0,0.999796
1,11834050,0.0,0.999850
2,11834051,0.0,0.999542
3,11834052,0.0,0.998738
4,11834053,0.0,0.998839


In [ ]:
df3.head() # This is what df should look like

,block_number,MEV_category,gas_used_ratio
0,11834049,4.0,0.999796
1,11834050,0.0,0.999850
2,11834051,0.0,0.999542
3,11834052,0.0,0.998738
4,11834053,0.0,0.998839


### Adding the price volitility

In [ ]:
infura_url = 'https://mainnet.infura.io/v3/285030c0cfe148938e2e033f8f2e59b0' # mu free archive infura node

In [ ]:
web3_infura = Web3(Web3.HTTPProvider(infura_url)) # instantiating a web3 instance

In [ ]:
contract_address = '0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419'
contract_abi = '[{"inputs":[{"internalType":"address","name":"_aggregator","type":"address"},{"internalType":"address","name":"_accessController","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"int256","name":"current","type":"int256"},{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"updatedAt","type":"uint256"}],"name":"AnswerUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":true,"internalType":"address","name":"startedBy","type":"address"},{"indexed":false,"internalType":"uint256","name":"startedAt","type":"uint256"}],"name":"NewRound","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"inputs":[],"name":"acceptOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"accessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"aggregator","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_aggregator","type":"address"}],"name":"confirmAggregator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRound","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address payable","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint16","name":"","type":"uint16"}],"name":"phaseAggregators","outputs":[{"internalType":"contract AggregatorV2V3Interface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"phaseId","outputs":[{"internalType":"uint16","name":"","type":"uint16"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_aggregator","type":"address"}],"name":"proposeAggregator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"proposedAggregator","outputs":[{"internalType":"contract AggregatorV2V3Interface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"proposedGetRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"proposedLatestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_accessController","type":"address"}],"name":"setController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]'

In [ ]:
ChainlinkTokenContract = web3_infura.eth.contract(contract_address, abi=contract_abi) # instantiate an instance of the Chainlink ETH-USD Price Oracle contract

In [ ]:
display(df4)

,block_number,MEV_category,gas_used_ratio
0,11834049,4.0,0.999796
1,11834050,0.0,0.999850
2,11834051,0.0,0.999542
3,11834052,0.0,0.998738
4,11834053,0.0,0.998839
...,...,...,...
39995,11874044,0.0,0.999181
39996,11874045,0.0,0.999162
39997,11874046,0.0,0.999377
39998,11874047,0.0,0.998756


In [ ]:

df4['eth_price'] = df4['block_number'].apply(lambda x: ChainlinkTokenContract.functions.latestRoundData().call(block_identifier=x)[1])

HTTPError: 429 Client Error: Too Many Requests for url: https://mainnet.infura.io/v3/285030c0cfe148938e2e033f8f2e59b0

In [2]:
df4 = pd.read_csv('..\Ethdata\MEVcategorywgandprice.csv')  # use this to load in prior CSV
df4.head()

,Unnamed: 0,block_number,MEV_category,gas_used_ratio,eth_price
0,0,11834049,4,0.999796,173857000000
1,1,11834050,0,0.999850,173857000000
2,2,11834051,0,0.999542,173857000000
3,3,11834052,0,0.998738,173857000000
4,4,11834053,0,0.998839,173857000000


In [3]:
df4['pricevol'] = df4['eth_price'].rolling(50).std()

df4.dropna()

df4.drop(['eth_price'],axis=1,inplace=True)

df4.head()
df4.tail()

,Unnamed: 0,block_number,MEV_category,gas_used_ratio,pricevol
5201,5201,11839250,0,0.998571,3.312210e+08
5202,5202,11839251,0,0.996847,3.232888e+08
5203,5203,11839252,0,0.996925,3.147701e+08
5204,5204,11839253,0,0.999256,3.056032e+08
5205,5205,11839254,0,0.999198,2.957280e+08


In [4]:
#Lastly we can save the data that has been created using this line. 
df4.to_csv('..\Ethdata\MEVcategorywgv1.csv') # how you would convert final dataframe to csv
